In [1]:
import pandas as pd
import numpy as np
from function_file import clean_names


# print(clean_df.isnull().any())
# print(clean_df.isnull().sum())

C:\Users\K\AppData\Local\Temp\ipykernel_19508\579319779.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
cake_header = pd.read_csv(r"data_in\cake_ingredient_code.csv")
raw_df = pd.read_csv(r"data_in\cake-ingredients-1961.csv")
# print(raw_df.isnull().sum())

In [3]:
# Data Model

clean_df = raw_df.copy()

# Pivot longer by Cake,
clean_df = (pd.melt(clean_df, id_vars = ["Cake"])
            .rename(columns = clean_names))
# clean_df = clean_df

# test = clean_df.set_index("variable").join(cake_header.set_index("code"))

df = pd.merge(clean_df, cake_header, left_on = 'variable', right_on = 'code')

#Create a unqiue list of ingredient
# df["ingredient"].drop_duplicates()
# df["cake"].drop_duplicates()

In [4]:
# 1.2.2 Analysis questions
# Which cake has the most cocoa in it?
# For sponge cake, how many cups of ingredients are used in total?
# How many ingredients are measured in teaspoons?
# Which cake has the most unique ingredients?
# Which ingredients are used only once?

In [5]:
# Which cake has the most cocoa in it?

(df.loc[(df["ingredient"] == "Cocoa") & (df["value"].notna())]
.sort_values(by=["value"], ascending = False)
.head(1))

,cake,variable,value,code,ingredient,measure
97,One Bowl Chocolate,CA,10.0,CA,Cocoa,tablespoon


In [6]:
# For sponge cake, how many cups of ingredients are used in total?

sponge_cups = df.copy()
sponge_cups.loc[(sponge_cups["cake"] == "Sponge") & (sponge_cups["measure"] == "cup")]

sponge_cups["value"].sum()

240.39999999999998

In [43]:
# How many ingredients are measured in teaspoons?

tspoon_count = cake_header.copy()

tspoon_count["measure"] = (tspoon_count["measure"]
                           .str.lower()
                           .str.strip()
                           .astype("category"))

ingred_measure_in_tspoon = (tspoon_count
                            .loc[tspoon_count['measure'] == "teaspoon"]
                            .drop_duplicates(subset="ingredient")
                            .shape[0])
ingred_measure_in_tspoon

8

In [61]:
# Which cake has the most unique ingredients?
cake_unique_ingred = df[["cake","ingredient","value"]].copy()
cake_unique_ingred = cake_unique_ingred.loc[cake_unique_ingred["value"].notna()].drop_duplicates(subset = "ingredient")

filtered = (cake_unique_ingred
.groupby(["cake"], sort = True)["ingredient"].count()).nlargest(1)
filtered

cake
Babas au Rhum            9
Name: ingredient, dtype: int64

In [87]:
# Which ingredients are used only once?

unique_ingred = df[["ingredient","value"]].copy()
unique_ingred = unique_ingred.loc[(unique_ingred["value"].notna()) & (unique_ingred["value"] != 0)]
filter = unique_ingred.groupby(["ingredient"])["value"].count().reset_index()
filter = filter.loc[filter["value"]==1]
filter

,ingredient,value
2,Bananas,1
7,Cream of tartar,1
9,Crushed Ice,1
10,Dried currants,1
11,Egg white,1
20,Nutmeg,1
21,Nuts,1
32,Zwiebach,1
